In [54]:
import pandas as pd
import numpy as np
import re
from keras.preprocessing import image
import matplotlib.pyplot as plt
from keras.applications.resnet50 import ResNet50,preprocess_input
from keras.models import Model
from keras.layers import *
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [55]:
def Read_text_file(path):
    with open(path) as f:
        caption=f.read()
        f.close()
    return caption

In [56]:
import os

In [57]:
os.listdir('../input/flickr8k/Flickr_Data/Flickr_Data/')

In [58]:
captions=Read_text_file('../input/flickr8k/Flickr_Data/Flickr_Data/Flickr_TextData/Flickr8k.token.txt')

In [59]:
# captions

In [60]:
len(captions.split('\n'))

In [61]:
captions=captions.split('\n')[:-1]

In [62]:
len(captions)

In [63]:
a=captions[1000]

In [64]:
a

In [65]:
a=a.split('\t')[0][:-6]

In [66]:
a

In [67]:
cap_dict={}

In [68]:
for i in captions:
    a,b=i.split('\t')
    a=a[:-6]
    if cap_dict.get(a) is None:
        cap_dict[a]=[]
    cap_dict[a].append(b)

In [69]:
len(cap_dict)

In [70]:
cap_dict[a]

In [71]:
# np.save('description_file.npy',cap_dict)

In [72]:
def sentence_cleaning(sentence):
    sentence=sentence.lower()
    sentence=re.sub('[^a-z]+',' ',sentence)
    sentence=sentence.split()
    sentence=[s for s in sentence if len(s)>1]
    sentence=' '.join(sentence)
    return(sentence)
    

In [73]:
z='A black and brown dog is laying on a @white shaggy carpet having number # 79 .'

In [74]:
sentence_cleaning(z)

### Cleaning every sentence of dictionary

In [75]:
for key in cap_dict:
    for j in range(len(cap_dict[key])):
        cap_dict[key][j]=sentence_cleaning(cap_dict[key][j])

In [76]:
cap_dict[a]

In [77]:
# np.save('description_file_2.npy',cap_dict)

In [78]:
word_dic={}

In [79]:
for i in cap_dict:
    for j in cap_dict[i]:
        l=j.split()
        for k in l:
            if k not in word_dic:
                word_dic[k]=1
            else:
                word_dic[k]+=1
    

In [80]:
len(word_dic)

In [81]:
len(word_dic)

In [82]:
final_words=[x for x in word_dic if word_dic[x]>10]

In [83]:
len(final_words)

In [84]:
 final_words

In [85]:
for i in cap_dict:
    for j in range(len(cap_dict[i])):
        cap_dict[i][j]='startseq '+cap_dict[i][j]+' endseq'

In [86]:
cap_dict[a]

In [87]:
with open('../input/flickr8k/Flickr_Data/Flickr_Data/Flickr_TextData/Flickr_8k.trainImages.txt') as f:
    train_img=f.read()

In [88]:
train_img_list=[x.split('.')[0] for x in train_img.split('\n')[:-1]]

In [89]:
train_img_list

In [90]:
train_description={}

In [91]:
for i in train_img_list:
    train_description[i]=cap_dict[i]

In [92]:
a

In [93]:
img=image.load_img('../input/flickr8k/Flickr_Data/Flickr_Data/Images/106490881_5a2dd9b7bd.jpg')

In [94]:
img=image.img_to_array(img)

In [95]:
img.shape

In [96]:
plt.imshow((img/255)*1)

## Creating model 

In [97]:
model=ResNet50(weights='imagenet',input_shape=(224,224,3))

In [98]:
model.summary()

In [100]:
new_model=Model(model.input,model.layers[-2].output)

In [101]:
def preprocess_img(path):
    img=image.load_img(path,target_size=(224,224,3))
    img=image.img_to_array(img)
    img=img.reshape(1,224,224,3)
    img=preprocess_input(img)#mormalizing the img
    return img

In [102]:
img=preprocess_img('../input/flickr8k/Flickr_Data/Flickr_Data/Images/106490881_5a2dd9b7bd.jpg')

In [103]:
plt.imshow(img[0])

In [104]:
def encode_img(path):
    img=preprocess_img(path)
    feature_vector=new_model.predict(img)
    feature_vector=feature_vector.reshape((-1,))
    return feature_vector
    

In [105]:
train_description['1244140539_da4804d828']

In [106]:
encoded_img_dic={}
s=0
for i in train_description:
    path='../input/flickr8k/Flickr_Data//Flickr_Data/Images/'+i+'.jpg'
    encoded_img_dic[i]=encode_img(path)
    s+=1
    if s%100==0:
        print(s)

In [107]:
np.save('encoded_img_dic.npy',encoded_img_dic)

In [108]:
!ls -l --b=M  ./encoded_img_dic.npy | cut -d " " -f5

## data preprocessing for captions

In [113]:
s=1
word_to_idx={}
idx_to_word={}
for i in final_words:
    word_to_idx[i]=s
    idx_to_word[s]=i
    s+=1

In [114]:
len(word_to_idx)

In [115]:
idx_to_word[1845]

In [116]:
### Two Special words
word_to_idx['startseq']=1846
word_to_idx['endseq']=1847
idx_to_word[1846]='startseq'
idx_to_word[1847]='endseq'

In [117]:
len(word_to_idx)

In [118]:
vocab_size=len(word_to_idx)+1 # adding one for 0 because that will also in our vector

In [119]:
vocab_size

In [120]:
sen_len=[]
for i in train_description:
    for sen in train_description[i]:
        sen_len.append(len(sen.split()))

In [121]:
len(sen_len)

In [122]:
plt.hist(sen_len,bins=20)

In [123]:
sen_len[0]

In [124]:
max_len=20

## Custom Data Generator

In [125]:
def data_generator(train_description,vocab_size,word_to_idx,encoded_img_dic,max_len,batch_size):
    X1,X2,y=[],[],[]
    n=0
    while True:
        for key,desc_list in train_description.items():
            n+=1
            encoding_of_photo=encoded_img_dic[key]
            for desc in desc_list:
                seq=[word_to_idx[i] for i in desc.split() if i in word_to_idx]
                for i in range(1,len(seq)):
                    xi=seq[0:i]
                    yi=seq[i]
                    
                    xi=pad_sequences([xi],maxlen=max_len,value=0,padding='post')[0]
                    yi=to_categorical([yi],num_classes=vocab_size)[0]
                    
                    
                    X1.append(encoding_of_photo)
                    X2.append(xi)
                    y.append(yi)
                if n==batch_size:
                    yield [np.array(X1),np.array(X2)],np.array(y)
                    X1,X2,y=[],[],[]
                    n=0
                    
                

## Saving and loading a dictionary using numpy

## Model Architecture

In [130]:
encoded_img_dic['1244140539_da4804d828'].shape

In [131]:
## For images
input_img_features=Input(shape=(2048,))
inp_img1=Dropout(0.3)(input_img_features)
inp_img2=Dense(256,activation='relu')(inp_img1)


In [132]:
vocab_size

In [133]:
input_captions=Input(shape=(max_len,))
inp_cap1=Embedding(input_dim=vocab_size,output_dim=50,mask_zero=True)(input_captions)
inp_cap2=Dropout(0.3)(inp_cap1)
inp_cap3=LSTM(256)(inp_cap2)

In [134]:
decoder1=add([inp_img2,inp_cap3])
decoder2=Dense(256,activation='relu')(decoder1)
outputs=Dense(vocab_size,activation='softmax')(decoder2)

In [135]:
actual_model=Model(inputs=[input_img_features,input_captions],outputs=outputs)


In [136]:
actual_model.summary()

In [137]:
actual_model.layers[2].output

In [138]:
actual_model.layers[2].set_weights([embedding_matrix])
actual_model.layers[2].trainable=False

In [142]:
actual_model.compile(loss='categorical_crossentropy',optimizer='adam')

## Training the Model

In [143]:
epochs=20
batch_size=3
steps=len(train_description)//batch_size

In [144]:
def train():
    for i in range(epochs):
        generator=data_generator(train_description,vocab_size,word_to_idx,encoded_img_dic,max_len,batch_size)
        actual_model.fit_generator(generator,epochs=1,steps_per_epoch=steps,verbose=1)
        actual_model.save('./Models/model'+str(i)+'.h5')
        

In [145]:
os.mkdir('Models')

In [146]:
train()

In [154]:
max_len

In [155]:
def predict_caption(img):
    img=img.reshape(1,224,224,3)
    img=preprocess_input(img)
    feature_vector=new_model.predict(img)
    feature_vector=feature_vector.reshape((1,2048,1))
    in_text='startseq'
    for i in range(max_len):
        seq=[word_to_idx[w] for w in in_text.split() if w in word_to_idx]
        seq=pad_sequences([seq],maxlen=max_len,padding='post')
        y_pred=actual_model.predict([feature_vector,seq])
        y_pred=y_pred.argmax()
        word=idx_to_word[y_pred]
        in_text+=' '+word
        
        if word=='endseq':
            break
    final_caption=in_text.split()[1:-1]
    final_caption=' '.join(final_caption)
    return final_caption

In [156]:
img=image.load_img('../input/flickr8k/Flickr_Data/Flickr_Data/Images/1007320043_627395c3d8.jpg',target_size=(224,224,3))
img=image.img_to_array(img)

In [157]:
plt.imshow(img/255)

In [158]:
predict_caption(img)

In [159]:
x=0
for i in os.listdir('../input/flickr8k/Flickr_Data/Flickr_Data/Images/'):
    i='../input/flickr8k/Flickr_Data/Flickr_Data/Images/'+i
    img=image.load_img(i,target_size=(224,224,3))
    img=image.img_to_array(img)
    plt.imshow(img/255)
    plt.show()
    print(predict_caption(img))
    x+=1
    if x>20:
        break
    

In [153]:
ls -l --b=M  ./Models/model15.h5 | cut -d " " -f5